In [1]:
import re
import numpy as np
import pandas as pd
import calendar

In [37]:
df = pd.read_csv('All Transactions over $100K in 3 Counties in 2025 - ALL_Three.csv')

In [38]:
df2 = pd.read_csv('All Transactions over $100K in 3 Counties in 2025 - ALL_Three.csv')

In [39]:
df2['PHY_CITY'].unique()

array(['Unincorporated County', 'POMPANO BEACH', 'WEST PALM BEACH',
       'Miami Beach', 'LAUDERDALE BY THE SEA', 'DEERFIELD BEACH',
       'SUNRISE', 'BOYNTON BEACH', 'BOCA RATON', 'Opa-locka',
       'PEMBROKE PARK', 'TAMARAC', 'Homestead', 'PALM BEACH GARDENS',
       'DAVIE', 'LAKE WORTH', 'MARGATE', 'Doral', 'COCONUT CREEK',
       'LIGHTHOUSE POINT', 'LAUDERHILL', 'LAUDERDALE LAKES',
       'FORT LAUDERDALE', 'DANIA BEACH', 'PEMBROKE PINES',
       'HALLANDALE BEACH', 'DELRAY BEACH', 'GREENACRES', nan,
       'PALM SPRINGS', 'ROYAL PALM BEACH', 'Miami', 'Hialeah',
       'North Miami', 'Aventura', 'Sunny Isles Beach', 'Palmetto Bay',
       'JUPITER', 'COOPER CITY', 'Miami Gardens', 'HOLLYWOOD',
       'Cutler Bay', 'CORAL SPRINGS', 'UNINCORPORATED', 'PLANTATION',
       'NORTH LAUDERDALE', 'North Miami Beach', 'NORTH PALM BEACH',
       'OAKLAND PARK', 'LAKE PARK', 'MIRAMAR', 'WEST PARK', 'West Miami',
       'WILTON MANORS', 'Miami Lakes', 'Miami Shores', 'HILLSBORO BEACH',
  

In [40]:
x = df2[df2['PHY_CITY'] == 'Bal Harbour']
x.MICRO.value_counts()

MICRO
Condominiums                          48
Hotels, motels                         4
Cooperatives                           1
Multi-family - fewer than 10 units     1
Name: count, dtype: int64

In [41]:
x = df2[df2['PHY_CITY'] == 'Miami']
x.MICRO.value_counts()

MICRO
Condominiums                                                                                                                                                                                                                                                            1082
Single Family                                                                                                                                                                                                                                                            444
Multi-family - fewer than 10 units                                                                                                                                                                                                                                       162
Office buildings, non-professional service buildings, multi-story                                                                                                                          

In [3]:
df['SALE_PRC1_CLEAN'] = df.SALE_PRC1.str.replace('$','').replace(',','',regex=True).astype(int)

In [4]:
# --- Setup ---
keep_labels = [
    'Single Family',
    'Condominiums',
    'Vacant Residential',
    'Cooperatives',
    'Mobile Homes',
]

In [5]:
# Case-insensitive exact-keep map (normalize hyphen/spaces for Single Family)
keep_norm = {k.lower().replace('-', ' ').strip(): k for k in keep_labels}

# Commercial patterns (covers common variants)
commercial_pat = re.compile(
    r'(store|office|professional\s+service(?:s)?(?:\s+(?:building|bldg))?|'
    r'multi[\s-]?family|multifamily|hotel|restaurant)s?',
    flags=re.IGNORECASE
)

In [6]:
# --- Normalize MICRO ---
micro = df['MICRO'].astype('string')
micro_norm = micro.str.strip()

# Start with 'Other'
bucket = pd.Series('Other', index=df.index, dtype='string')

# Unknown/empty
bucket = bucket.mask(micro_norm.isna() | (micro_norm == ''), 'Unknown')

# Exact keep (case-insensitive; also accept "Single-Family")
micro_exact_norm = micro_norm.str.lower().str.replace('-', ' ', regex=False)
keep_mask = micro_exact_norm.isin(keep_norm.keys())
bucket = bucket.mask(keep_mask, micro_exact_norm.map(keep_norm))

# Commercial via regex
commercial_mask = micro_norm.str.contains(commercial_pat, na=False)
bucket = bucket.mask(commercial_mask, 'Commercial')

df['MICRO_bucket'] = bucket

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_30244/618607014.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  commercial_mask = micro_norm.str.contains(commercial_pat, na=False)


In [7]:
# --- Aggregate ---
df['SALE_PRC1_CLEAN'] = pd.to_numeric(df['SALE_PRC1_CLEAN'], errors='coerce')
groupby_cols = ['MICRO_bucket', 'SALE_MO1', 'PHY_CITY']

agg_df = (
    df.groupby(['MICRO_bucket', 'SALE_MO1', 'PHY_CITY'], dropna=False)
      .agg(
          avg_price=('SALE_PRC1_CLEAN', 'mean'),
          median_price=('SALE_PRC1_CLEAN', 'median'),
          valid_n=('SALE_PRC1_CLEAN', 'count'),              # non-NaN count
          total_dollar_volume=('SALE_PRC1_CLEAN', 'sum')     # sum of dollars
      )
      .reset_index()
      .query("valid_n >= 5")
      .rename(columns={'valid_n':'n'})
)


In [8]:
# Optional polish
agg_df['avg_price'] = agg_df['avg_price'].round(2)
agg_df['median_price'] = agg_df['median_price'].round(2)
agg_df = agg_df.sort_values(groupby_cols)

In [9]:
agg_df.MICRO_bucket.value_counts()

MICRO_bucket
Single Family         341
Condominiums          316
Vacant Residential     84
Commercial             77
Other                  57
Cooperatives           21
Mobile Homes            9
Name: count, dtype: Int64

In [10]:
agg_df.SALE_MO1.value_counts()

SALE_MO1
4    187
3    183
5    169
2    166
1    161
6     39
Name: count, dtype: int64

In [11]:
agg_df

,MICRO_bucket,SALE_MO1,PHY_CITY,avg_price,median_price,n,total_dollar_volume
2,Commercial,1,BOCA RATON,1313000.00,1185000.0,5,6565000
6,Commercial,1,CORAL SPRINGS,847812.50,645000.0,8,6782500
7,Commercial,1,Coral Gables,1426000.00,1550000.0,5,7130000
9,Commercial,1,DANIA BEACH,714000.00,630000.0,5,3570000
12,Commercial,1,DELRAY BEACH,1229000.00,1340000.0,5,6145000
...,...,...,...,...,...,...,...
1756,Vacant Residential,5,PEMBROKE PINES,680833.33,678000.0,6,4085000
1757,Vacant Residential,5,POMPANO BEACH,472414.29,467000.0,7,3306900
1762,Vacant Residential,5,Unincorporated County,1878454.10,829000.0,61,114585700
1765,Vacant Residential,5,WESTLAKE,590733.77,607778.5,48,28355221


In [12]:
# Pick the actual column name you have
month_col = 'SALE_MO1' if 'SALE_MO1' in agg_df.columns else 'SALE_M01'

# Build 1..12 → "January".. "December"
month_map = {i: calendar.month_name[i] for i in range(1, 13)}

# Make a new column with full month names
agg_df[month_col + '_NAME'] = (
    pd.to_numeric(agg_df[month_col], errors='coerce')     # "01" → 1, keeps NaN if missing
      .astype('Int64')                                # nullable integer dtype
      .map(month_map)                                 # 1 → "January", NaN stays NaN
)

# (Optional) If you want to REPLACE the original column instead:
# df[month_col] = pd.to_numeric(df[month_col], errors='coerce').astype('Int64').map(month_map)


In [13]:
agg_df['PHY_CITY'] = agg_df['PHY_CITY'].str.title()

In [14]:
agg_df = agg_df.drop(columns='SALE_MO1')

In [15]:
agg_df[(agg_df['PHY_CITY'] == 'Miami') & (agg_df['SALE_MO1_NAME'] == 'June')]

,MICRO_bucket,PHY_CITY,avg_price,median_price,n,total_dollar_volume,SALE_MO1_NAME
689,Condominiums,Miami,995857.14,437000.0,14,13942000,June


In [16]:
agg_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 905 entries, 2 to 1767
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   MICRO_bucket         905 non-null    string 
 1   PHY_CITY             898 non-null    object 
 2   avg_price            905 non-null    float64
 3   median_price         905 non-null    float64
 4   n                    905 non-null    int64  
 5   total_dollar_volume  905 non-null    int64  
 6   SALE_MO1_NAME        905 non-null    object 
dtypes: float64(2), int64(2), object(2), string(1)
memory usage: 56.6+ KB


In [17]:
agg_df.to_csv('agg_df_cards.csv')